#7.3.3 문자 단위 RNN 실습 2가지
- RNN의 입출력의 단위가 단어 레벨이 아닌 문자 레벨로 하여 RNN을 구현하면 이를 문자 단위 RNN이라고 함
  - RNN의 구조 자체가 달리진 것이 아닌 입 출력의 단위가 문자로 바뀌었을 뿐임

###문자 단위 RNN(Char RNN)

[1] 필요 도구 import

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

[2] 훈련 데이터 전처리
- apple을 입력 -> pple!룰 출력하도록 구현
- 입력 데이터와 레이블 데이터에 대한 문자 집합을 만들도록 구현

In [2]:
input_str = 'apple'
label_str = 'pple!'
char_vocab = sorted(list(set(input_str+label_str)))
vocab_size = len(char_vocab)
print('문자 집합의 크기: {}'.format(vocab_size))

문자 집합의 크기: 5


In [3]:
input_size=vocab_size # 입력의 크기는 문자 집합의 크기
hidden_size = 5
output_size = 5
learning_rate = 0.1

In [4]:
char_to_index = dict((c, i) for i, c, in enumerate(char_vocab)) # 문자에 고유한 정수 인덱스 부여
print(char_to_index)

{'!': 0, 'a': 1, 'e': 2, 'l': 3, 'p': 4}


In [5]:
index_to_char={}
for key, value in char_to_index.items():
  index_to_char[value] = key
print(index_to_char)

{0: '!', 1: 'a', 2: 'e', 3: 'l', 4: 'p'}


In [6]:
x_data = [char_to_index[c] for c in input_str]
y_data = [char_to_index[c] for c in label_str]
print(x_data)
print(y_data)

[1, 4, 4, 3, 2]
[4, 4, 3, 2, 0]


In [7]:
# 배치 차원 추가
# 텐서 연산인 unsqueeze(0)를 통해 해결할 수도 있었음.
x_data = [x_data]
y_data = [y_data]
print(x_data)
print(y_data)

[[1, 4, 4, 3, 2]]
[[4, 4, 3, 2, 0]]


In [8]:
x_one_hot = [np.eye(vocab_size)[x] for x in x_data]
print(x_one_hot)

[array([[0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0.]])]


In [9]:
X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)

/tmp/ipython-input-2348034151.py:1: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /pytorch/torch/csrc/utils/tensor_new.cpp:253.)
  X = torch.FloatTensor(x_one_hot)


In [10]:
print('훈련 데이터의 크기: {}'.format(X.shape))
print('레이블의 크기: {}'.format(Y.shape))

훈련 데이터의 크기: torch.Size([1, 5, 5])
레이블의 크기: torch.Size([1, 5])


[3] 모델 구현하기
- fc는 완전 연결층을 의미하며, 출력층으로 사용됨

In [11]:
class Net(torch.nn.Module):
  def __init__(self, input_size, hidden_size, output_size):
    super(Net, self).__init__()
    self.rnn = torch.nn.RNN(input_size, hidden_size, batch_first=True)  # RNN  셀 구현
    self.fc = torch.nn.Linear(hidden_size, output_size, bias=True)  # 출력층 구현

  def forward(self, x): # 구현한 RNN 셀과 출력층을 연결
    x, _status = self.rnn(x)
    x = self.fc(x)
    return x

In [12]:
net = Net(input_size, hidden_size, output_size)

In [14]:
outputs = net(X)
print(outputs.shape)  # 3차원 텐서

torch.Size([1, 5, 5])


In [15]:
print(Y.shape)
print(Y.view(-1).shape)

torch.Size([1, 5])
torch.Size([5])


In [16]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), learning_rate)

In [18]:
for i in range(100):
  optimizer.zero_grad()
  outputs = net(X)
  loss = criterion(outputs.view(-1, input_size), Y.view(-1))  # view를 하는 이유는 Batch 차원 제거를 위해
  loss.backward() # 기울기 계산
  optimizer.step() # 아까 optimizer 선언 시 넣어둔 파라미터 업데이트
  # 아래 세 줄은 모델이 실제 어떻게 예측했는지를 확인하기 위한 코드.
  result = outputs.data.numpy().argmax(axis=2)  # 최종 예측값인 각 time-step 별 5차원 벡터에 대해서 가장 높은 값의 인덱스를 선택
  result_str = ''.join([index_to_char[c] for c in np.squeeze(result)])
  print(i, "loss: ", loss.item(), "prediction: ", result, "true Y: ", y_data, "prediction str: ", result_str)

0 loss:  1.6503998041152954 prediction:  [[2 2 2 2 2]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  eeeee
1 loss:  1.3086830377578735 prediction:  [[3 3 3 2 0]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  llle!
2 loss:  1.0701619386672974 prediction:  [[4 4 4 2 0]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  pppe!
3 loss:  0.8493860363960266 prediction:  [[4 4 4 2 0]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  pppe!
4 loss:  0.6647893190383911 prediction:  [[4 4 4 2 0]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  pppe!
5 loss:  0.531540036201477 prediction:  [[4 4 4 2 0]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  pppe!
6 loss:  0.4299473166465759 prediction:  [[4 4 4 2 0]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  pppe!
7 loss:  0.33964022994041443 prediction:  [[4 4 4 2 0]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  pppe!
8 loss:  0.2567402422428131 prediction:  [[4 4 3 2 0]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  pple!
9 loss:  0.18563823401927948 prediction:  [[4 4 3 2 0]]

### 더 많은 데이터로 학습한 문자 단위 RNN(Char RNN)


[1] 필요 도구 임포트

In [19]:
import torch
import torch.nn as nn
import torch.optim as optim

[2] 훈련 데이터 전처리하기

In [22]:
sentence = ("if you want to build a ship, don't drum up preple together to"
  "collect wood and don't assign them tasks and work, but rather"
  "teach them to long for the endless immensity of the sea.")

In [23]:
char_set = list(set(sentence))  # 중복을 제거한 문자 집합 생성
char_dic = {c: i for i, c in enumerate(char_set)} # 각 문자에 정수 인코딩

In [24]:
print(char_dic) # 공백도 여기서는 하나의 원소

{'u': 0, 'h': 1, 'n': 2, 'k': 3, 'l': 4, 'e': 5, 'w': 6, "'": 7, 'f': 8, '.': 9, 'p': 10, 'o': 11, 'm': 12, 'y': 13, ' ': 14, 't': 15, 'a': 16, ',': 17, 'b': 18, 'r': 19, 's': 20, 'c': 21, 'g': 22, 'i': 23, 'd': 24}


In [25]:
dic_size = len(char_dic)
print('문자 집합의 크기: {}'.format(dic_size))

문자 집합의 크기: 25


In [28]:
# 하이퍼 파라미터 설정
hidden_size = dic_size
sequence_length = 10   # 임의 숫자 지정
learning_rate = 0.1

In [29]:
# 데이터 구성
x_data = []
y_data = []

for i in range(0, len(sentence) - sequence_length) :
  x_str = sentence[i:i + sequence_length]
  y_str = sentence[i +1: i + sequence_length + 1]
  print( i, x_str, '->', y_str)

  x_data.append([char_dic[c]for c in x_str])  # x str to index
  y_data.append([char_dic[c] for c in y_str]) # y str to index

0 if you wan -> f you want
1 f you want ->  you want 
2  you want  -> you want t
3 you want t -> ou want to
4 ou want to -> u want to 
5 u want to  ->  want to b
6  want to b -> want to bu
7 want to bu -> ant to bui
8 ant to bui -> nt to buil
9 nt to buil -> t to build
10 t to build ->  to build 
11  to build  -> to build a
12 to build a -> o build a 
13 o build a  ->  build a s
14  build a s -> build a sh
15 build a sh -> uild a shi
16 uild a shi -> ild a ship
17 ild a ship -> ld a ship,
18 ld a ship, -> d a ship, 
19 d a ship,  ->  a ship, d
20  a ship, d -> a ship, do
21 a ship, do ->  ship, don
22  ship, don -> ship, don'
23 ship, don' -> hip, don't
24 hip, don't -> ip, don't 
25 ip, don't  -> p, don't d
26 p, don't d -> , don't dr
27 , don't dr ->  don't dru
28  don't dru -> don't drum
29 don't drum -> on't drum 
30 on't drum  -> n't drum u
31 n't drum u -> 't drum up
32 't drum up -> t drum up 
33 t drum up  ->  drum up p
34  drum up p -> drum up pr
35 drum up pr -> rum up pre
36

In [30]:
print(x_data[0])
print(y_data[0])

[23, 8, 14, 13, 11, 0, 14, 6, 16, 2]
[8, 14, 13, 11, 0, 14, 6, 16, 2, 15]


In [32]:
x_one_hot = [np.eye(dic_size)[x] for x in x_data]  # x데이터는 원-핫 인코딩
X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)

In [33]:
print('훈련 데이터의 크기: {}'.format(X.shape))
print('레이블의 크기: {}'.format(Y.shape))

훈련 데이터의 크기: torch.Size([168, 10, 25])
레이블의 크기: torch.Size([168, 10])


In [34]:
print(X[0])

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,

In [35]:
print(Y[0])

tensor([ 8, 14, 13, 11,  0, 14,  6, 16,  2, 15])


[3] 모델 구현하기

In [39]:
class Net(torch.nn.Module):
  def __init__(self, input_dim, hidden_dim, layers):   # 현재 hidden_size는 dic_size와 같음.
    super(Net, self).__init__()
    self.rnn = torch.nn.RNN(input_dim, hidden_dim, num_layers = layers, batch_first = True)
    self.fc = torch.nn.Linear(hidden_dim, hidden_dim, bias=True)
  def forward(self, x):
    x, _status = self.rnn(x)
    x = self.fc(x)
    return x

In [40]:
net = Net(dic_size, hidden_size, 2) # 이번에는 층을 두 개 쌓습니다.

In [41]:
cirterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), learning_rate)

In [42]:
output = net(X)
print(outputs.shape)  # 3차원 텐서

torch.Size([1, 5, 5])


In [44]:
print(outputs.view(-1, dic_size).shape) #2차원 텐서로 변환.

torch.Size([1, 25])


In [45]:
print(Y.shape)
print(Y.view(-1).shape)

torch.Size([168, 10])
torch.Size([1680])


In [46]:
for i in range(100):
  optimizer.zero_grad()
  outputs = net(X)  # (170, 10, 25) 크기를 가진 텐서르 매 에포크마다 모델의 입력으로 사용
  loss = criterion(outputs.view(-1, dic_size), Y.view(-1))
  loss.backward()
  optimizer.step()

  # results의 텐서 크기는 (170,10)
  results = outputs.argmax(dim=2)
  predict_str = ""
  for j, result in enumerate(results):
    if j == 0:  # 처음에는 예측 결과를 전부 가져오지만
      predict_str += ''.join([char_set[t] for t in result])
    else: # 그 다음에는 마지막 글자만 반복 추가
     predict_str += char_set[result[-1]]
  print(predict_str)

noooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooo
ttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttt
eee ee  e  e e eeeeee eeee ee eee e eeeeeee ee ee  eeeeeeee eeeeeeeee eeeeee eeeeeee eeeee eeeeeeeeee  ee eeeeee eeeee eeeeeeeee eeeeee eee eeeeeeee ee eee eeeeeeeeee ee eee eee
 rbo ..o .  rfr .fr .rrr .rnrr ..ffr r rf.t ..  ff ffhfff frrfrn.f frrffrrf.  r.r rfr  nnf. frffr. h.f ffrrff f.r rrr.. . f rff.rn. rr.rh..  rf  rf.  ff r.f rf  .nffrrrnrh..  fr
tot tt  tt t tt pt pp t ttoit t t t tp t  toit t tu pptptpp tt it tup tp t tut tpt t tptp t pt t tt tp t t tpt t t t tt t t tputott tottott t tpt tttuttt ptt tpt pp tt cp pt t i
to o t  ttoto toio tt to to tott ttontoto to t to o ttitt to toio toto toto otottt tott at o t to t itototonio